In [8]:
# !git clone https://github.com/akshitv2/VAE-latent-space-experiment.git
# %cd /content/VAE-latent-space-experiment

fatal: destination path 'VAE-latent-space-experiment' already exists and is not an empty directory.
/content/VAE-latent-space-experiment


In [3]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os
from tqdm import tqdm

from experiments.Checkpointing import save_checkpoint
from models.VAE import VAE
from modules.Losses import vae_loss
from modules.SaveOutputs import save_reconstructions, save_samples

dataset_dir: str = "./data/raw"
out_dir: str = "./outputs/"
batch_size: int = 512
latent_dim: int = 512
checkpoint_dir = "./experiments/checkpoints"
epochs: int = 10
lr: float = 3e-4
beta: float = 0.5

torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((64, 64)),  # resize to 224x224
    transforms.ToTensor()  # convert to tensor & scale to [0,1]
])


In [10]:
dataset = datasets.ImageFolder(root="G:\Temp", transform=transform)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

print("Loaded datasets, number of samples: ", len(dataset))

# Model & Optimizer
model = VAE(latent_dim=latent_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


global_step = 0
os.makedirs(out_dir, exist_ok=True)

Loaded datasets, number of samples:  202599


In [11]:
train_mode = True
current_epoch = 0
epochs = 2
beta = 0.001
beta_anneal_factor = 1.1

In [20]:
training_loss_tracker = []

In [ ]:
epochs = 2000
if train_mode:
    for epoch in range(current_epoch + 1, current_epoch + epochs + 1):
        model.train()
        running_total = running_recon = running_kld = 0.0

        for batch_idx, (x, _) in enumerate(tqdm(train_loader), start=1):
            x = x.to(device)
            optimizer.zero_grad(set_to_none=True)
            logits, mean, logvar = model(x)
            loss = vae_loss(logits, x, mean, logvar, beta=beta)
            loss.total.backward()
            optimizer.step()

            running_total += loss.total.item()
            running_recon += loss.recon.item()
            running_kld += loss.kld.item()
        current_epoch += 1
        beta = min(0.8, beta * beta_anneal_factor)

        save_reconstructions(model, x, out_dir, current_epoch, device)
        n_train = len(train_loader.dataset)
        print(
            f"Epoch {epoch:02d} | total: {running_total / n_train:.4f} | "
            f"recon: {running_recon / n_train:.4f} | kld: {running_kld / n_train:.4f} | "
            f"beta: {beta:.4f}"
        )
        if(epoch % 10) == 0:
            training_loss_tracker.append([running_total / n_train, running_recon / n_train, running_kld / n_train, beta])

        if epoch % 100 == 0:
            save_checkpoint(model, optimizer, epoch, checkpoint_dir)

  0%|          | 0/396 [00:00<?, ?it/s]